<a href="https://colab.research.google.com/github/rk0803/Assignment3/blob/main/Assignment3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim 

In [3]:
torch.cuda.is_available()

True

In [4]:
def get_device():
  if torch.cuda.is_available():
    return torch.device('cuda')
  else:
    return torch.device('cpu')
device=get_device()
device

device(type='cuda')

In [5]:
def to_device(data,device):
  if isinstance(data,(list,tuple)):
      return[to_device(x,device) for x in data]
  return data.to(device,non_blocking=True)

In [6]:
from torch.utils.data import Dataset

In [7]:
class MySet(Dataset):
  def __init__(self,csv_file):
    self.data=pd.read_csv(csv_file)

  def __getitem__(self,index):
    r=self.data.iloc[index]
    label=torch.tensor(r.is_up_day,dtype=torch.long)
    sample=self.normalize(torch.tensor([r.open,r.high,r.low,r.close]))
    return sample, label

  def __len__(self):
    return len(self.data)

In [ ]:
# Training set of MNIST data set is created.

In [9]:
train_set=torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()])
)

In [10]:
train_loader=torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [11]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
   def __init__(self):
        super().__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution 
        #kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels to second layer and 12 output channels, 
        #5X5 convolution kernel
        self.conv2 = nn.Conv2d(6, 12, 5)
        # flattening the output to further use in fully connected layers
        self.fc1 = nn.Linear(192, 120) 
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10) 
        self.fc4 = nn.Linear(20,100)
        self.fc5 = nn.Linear(100,60)
        self.out = nn.Linear(60,19)

   def forward(self, x,y):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,kernel_size=2,stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,kernel_size=2,stride=2)
        x = x.reshape(-1,12*4*4) # flattening the other dimensions of output 
        #except the batch side
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        imgdig=x #image digit output is captured here
        x = torch.cat((x,y),dim=1) # Concatenation of one-hot vector of 
        #random digit which is the second input
        x = torch.tanh(self.fc4(x))
        x = torch.tanh(self.fc5(x))
        x = self.out(x)
        return x,imgdig


net = Net() # creating an instance of the NN created above
print(net) # to check out the parameters.

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
  (fc4): Linear(in_features=20, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=19, bias=True)
)


Get the number of correct labels outputted


In [13]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

To create the set of 60000 random digits in batche of size 100

In [14]:
import random
from random import randrange

In [15]:
import numpy as np
yr=np.array([[random.randrange(0,9,1) for j in range(100)] for i in range(600)])
yr=torch.as_tensor(yr)
yr=to_device(yr,device)

In [16]:
#training Neural Net with all batches
net=Net()
to_device(net,device)
trainloader=torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer=optim.SGD(net.parameters(),lr=0.01,momentum =0.9)
for epoch in range(50):
  total_loss=0
  total_correct=0
  i=0
  for batch in train_loader:
    batch=to_device(batch,device)
    y=yr[i] # pickup the first batch of random digits 
    i+=1
    y1=to_device(F.one_hot(y,10),device) # convert into one-hot vector notation
    images,labels=batch  #first batch from training set of image data set.
    sumlabint=y+labels   #output labels for the sum of two digits 
    preds,img1=net(images,y1)
    loss1=F.cross_entropy(img1,labels)
    loss2=F.cross_entropy(preds,sumlabint)
    loss=loss1+loss2
    optimizer.zero_grad()
    loss.backward()  #Calculate gradients
    optimizer.step() # Update weights
    total_loss += loss.item()
    total_correct+=(get_num_correct(img1,labels))
  print("epoch ", epoch,"total_correct: ", total_correct, "loss ",total_loss)   



epoch  0 total_correct:  39283 loss  1998.9191895723343
epoch  1 total_correct:  57964 loss  727.6856839060783
epoch  2 total_correct:  58591 loss  322.17051205039024
epoch  3 total_correct:  58898 loss  156.61982273310423
epoch  4 total_correct:  59111 loss  99.50963361561298
epoch  5 total_correct:  59256 loss  79.14698752947152
epoch  6 total_correct:  59366 loss  65.65596918854862
epoch  7 total_correct:  59475 loss  54.1232724301517
epoch  8 total_correct:  59530 loss  45.398660878650844
epoch  9 total_correct:  59598 loss  39.936515281908214
epoch  10 total_correct:  59652 loss  33.82143662078306
epoch  11 total_correct:  59654 loss  31.559487613849342
epoch  12 total_correct:  59685 loss  28.070366208441556
epoch  13 total_correct:  59725 loss  25.562164240982383
epoch  14 total_correct:  59757 loss  21.562899586977437
epoch  15 total_correct:  59782 loss  19.795338658615947
epoch  16 total_correct:  59782 loss  20.211533706169575
epoch  17 total_correct:  59850 loss  15.5032468

In [17]:
test_set=torchvision.datasets.MNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([transforms.ToTensor()])
)

In [18]:
yr=np.array([[random.randrange(0,9,1) for j in range(100)] for i in range(100)])
yr=torch.as_tensor(yr)
yr=to_device(yr,device)

In [19]:
testloader=torch.utils.data.DataLoader(test_set,batch_size=100)
total_correct_dig=0
total_correct_sum=0
i=0
for batch in testloader:
    batch=to_device(batch,device)
    images,labels=batch
    y=yr[i] # pick up the batch of random digits
    i+=1
    y1=to_device(F.one_hot(y,10),device) # convert it into one-vector
    sumlabint=y+labels   # Create the output labels for sum of image digit and random digit
    preds,img1=net(images,y1)
    total_correct_dig+=(get_num_correct(img1,labels))
    total_correct_sum+=(get_num_correct(preds,sumlabint))
print("total_correct_digits: ", total_correct_dig, "total_correct_sum: ", total_correct_sum) 



total_correct_digits:  9915 total_correct_sum:  9917
